In [34]:
'''If we are working with a really huge dataset, we can also use 98/1/1
portion for train/test set.
'''

'If we are working with a really huge dataset, we can also use 98/1/1\nportion for train/test set.\n'

In [43]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


In [44]:
#import dataset (comes with seaborn)
import seaborn as sns
iris = sns.load_dataset('iris')

#convert from panddas dataframe to tensor
data = torch.tensor(iris[iris.columns[0:4]].values).float()

#transform specifes to number
labels = torch.zeros(len(data), dtype=torch.long)
#labels[iris.species=='setosa'] = 0 #don't need!
labels[iris.species=='versicolor'] = 1
labels[iris.species=='virginica'] = 2

In [45]:
#Aside on using train_test_split

#create our fake dataset
fakedata = np.tile(np.array([1,2,3,4]),(10,1)) + np.tile(10*np.arange(1,11),(4,1)).T
fakelabels=np.arange(10)>4
print(fakedata),print(' ')
print(fakelabels)

[[ 11  12  13  14]
 [ 21  22  23  24]
 [ 31  32  33  34]
 [ 41  42  43  44]
 [ 51  52  53  54]
 [ 61  62  63  64]
 [ 71  72  73  74]
 [ 81  82  83  84]
 [ 91  92  93  94]
 [101 102 103 104]]
 
[False False False False False  True  True  True  True  True]


In [46]:
#use scikit-learn to split data
train_data, test_data, train_labels, test_labels = \
train_test_split(fakedata, fakelabels, test_size=.2)

#NOTE the third input parameter above
#This can be specified as test size or training size
#Be mindful of whcih parameter is writren!

#print out the sizes
print('Training data size:' + str(train_data.shape))
print('Test data size:' + str(test_data.shape))
print(' ')

#print out the train/test data
print('Training data:')
print(train_data)
print(' ')

print('Test data:')
print(test_data)

Training data size:(8, 4)
Test data size:(2, 4)
 
Training data:
[[ 61  62  63  64]
 [ 51  52  53  54]
 [ 11  12  13  14]
 [101 102 103 104]
 [ 41  42  43  44]
 [ 31  32  33  34]
 [ 21  22  23  24]
 [ 91  92  93  94]]
 
Test data:
[[71 72 73 74]
 [81 82 83 84]]


In [47]:
#a function that creates the ANN model

def createANewModel():
    
    #model architecture
    ANNiris = nn.Sequential(
        nn.Linear(4,64),  #input layer
        nn.ReLU(),        #activation unit
        nn.Linear(64,64),  #hidden layer
        nn.ReLU(),        #activation unit
        nn.Linear(64,3),   #output units
    )

    #loss function
    lossfun = nn.CrossEntropyLoss()
    
    #optimizer
    optimizer = torch.optim.SGD(ANNiris.parameters(),lr=.01)
    
    return ANNiris,lossfun,optimizer

In [58]:
#train the model

#global parameter
numepochs=200

def trainTheModel(trainProp):
    
    #initialize losses
    losses=torch.zeros(numepochs)
    trainAcc = []
    testAcc = []
    
    #loop over the epochs
    for epochi in range(numepochs):
        
        #seperate train from test data
        #Note1: unique split for each epoch !
        #Note2: here we specify the training size, not the testing size!
        X_train,X_test,y_train,y_test = train_test_split(data,labels,train_size=trainProp)
        #This is overfitting because the items that are in the test set
        #are likely to be in the training set in the next epoch.
        
        #forward pass and loss
        yHat = ANNiris(X_train)
        loss = lossfun(yHat,y_train)
        
        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #compute training accuracy
        trainAcc.append(100*torch.mean((torch.argmax(yHat,axis=1) == y_train).float()).item())
        
        #test accuracy:
        predlabels = torch.argmax(ANNiris(X_test),axis=1)
        testAcc.append(100*torch.mean((predlabels == y_test).float()).item())
        
    #function output:
    return trainAcc, testAcc
        

In [59]:
#Test the model by running it once
ANNiris,lossfun,optimizer = createANewModel()

#train the model
#Note: the input is the training proportion, not the test proportion!
trainAcc,testAcc = trainTheModel(.8)

In [60]:
#plot the results
fig = plt.figure(figsize=(10,5))

plt.plot(trainAcc,'ro-')
plt.plot(testAcc,'bs-')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend(['Train','Test'])
plt.show()

In [62]:
#Now for the experiment
trainSetSizes = np.linspace(.2,.95,10)

allTrainAcc = np.zeros((len(trainSetSizes),numepochs))
allTestAcc = np.zeros((len(trainSetSizes),numepochs))

for i in range(len(trainSetSizes)):
    
    #create a model
    ANNiris,lossfun,optimizer = createANewModel()
    
    #train the model
    trainAcc,testAcc = trainTheModel(trainSetSizes[i])
    
    #store the results
    allTrainAcc[i,:] = trainAcc
    allTestAcc[i,:] = testAcc

In [64]:
fig,ax = plt.subplots(1,2,figsize=(13,5))

ax[0].imshow(allTrainAcc,aspect='auto',
            vmin=50,vmax=90, extent=[0,numepochs,trainSetSizes[-1],trainSetSizes[0]])
ax[0].set_ylabel('epochs')
ax[0].set_ylabel('Training size proportion')
ax[0].set_title('Training accuracy')

p=ax[1].imshow(allTestAcc, aspect='auto',
              vmin=50, vmax=90, extent=[0,numepochs,trainSetSizes[-1],trainSetSizes[0]])
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Training size proportion')
ax[1].set_title("test accuracy")
fig.colorbar(p,ax=ax[1])

plt.show()

In [ ]:
'''
Here yellow means higher accuracy and blue means low accuarcy.x
'''